## 2.1　初识神经网络
一个神经网络的具体实例：使用Python的Keras库来学习手写数字分类。在这个例子中，我们要解决的问题是，将手写数字的灰度图像（28像素×28像素）划分到10个类别中（从0到9）。我们将使用MNIST数据集，它是机器学习领域的一个经典数据集，这个数据集包含60 000张训练图像和10 000张测试图像，由美国国家标准与技术研究院（National Institute of Standards and Technology，即MNIST中的NIST）在20世纪80年代收集而成。我们可以将“解决”MNIST问题看作深度学习的“Hello World”，用来验证我们的算法正在按预期运行。

**说明　在机器学习中，分类问题中的某个类别叫作类（class），数据点叫作样本（sample），与某个样本对应的类叫作标签（label）。**

### 代码清单1：加载Keras中的MNIST数据集

In [25]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images和train_labels组成了训练集，模型将从这些数据中进行学习。然后，我们在测试集（包括test_images和test_labels）上对模型进行测试。
图像被编码为NumPy数组，而标签是一个数字数组，取值范围是0～9。图像和标签一一对应。

#### 观察训练数据

In [26]:
train_images.shape

(60000, 28, 28)

In [27]:
len(train_labels)

60000

In [28]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

观察测试数据

In [29]:
test_images.shape

(10000, 28, 28)

In [30]:
len(test_labels)

10000

In [31]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

### 代码清单2　神经网络架构

工作流程如下：
* 首先，将训练数据（train_images和train_labels）输入神经网络；
* 然后，神经网络学习将图像和标签关联在一起；
* 最后，神经网络对test_images进行预测，我们来验证这些预测与test_labels中的标签是否匹配

In [32]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

神经网络的核心组件是层（layer）。你可以将层看成数据过滤器：进去一些数据，出来的数据变得更加有用。具体来说，层从输入数据中提取表示——我们期望这种表示有助于解决手头的问题。大多数深度学习工作涉及将简单的层链接起来，从而实现渐进式的数据蒸馏（data distillation）。深度学习模型就像是处理数据的筛子，包含一系列越来越精细的数据过滤器（也就是层）。

本例中的模型包含2个Dense层，它们都是密集连接（也叫全连接）的神经层。第2层（也是最后一层）是一个10路softmax分类层，它将返回一个由10个概率值（总和为1）组成的数组。每个概率值表示当前数字图像属于10个数字类别中某一个的概率。

### 代码清单3　编译步骤

在训练模型之前，我们还需要指定编译（compilation）步骤的3个参数。
1. 优化器（optimizer）：模型基于训练数据来自我更新的机制，其目的是提高模型性能。
2. 损失函数（loss function）：模型如何衡量在训练数据上的性能，从而引导自己朝着正确的方向前进。
3. 在训练和测试过程中需要监控的指标（metric）：本例只关心精度（accuracy），即正确分类的图像所占比例。

In [33]:
model.compile(optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

### 代码清单4　准备图像数据

在开始训练之前，我们先对数据进行预处理，将其变换为模型要求的形状，并缩放到所有值都在[0,1]区间。前面提到过，训练图像保存在一个uint8类型的数组中，其形状为(60000, 28, 28)，取值区间为[0, 255]。我们将把它变换为一个float32数组，其形状为(60000, 28 * 28)，取值范围是[0, 1]。

In [34]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32")/255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32")/255

### 代码清单5　拟合模型
准备开始训练模型。在Keras中，这一步是通过调用模型的fit方法来完成的——在训练数据上拟合（fit）模型

In [36]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0290 - accuracy: 0.9912
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0215 - accuracy: 0.9941
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0165 - accuracy: 0.9953
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0132 - accuracy: 0.9963
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0097 - accuracy: 0.9975


训练过程中显示了两个数字：一个是模型在训练数据上的损失值（loss），另一个是模型在训练数据上的精度（acc）。我们很快就在训练数据上达到了0.99（99.7%）的精度。

### 代码清单6　利用模型进行预测
现在我们得到了一个训练好的模型，可以利用它来预测新数字图像的类别概率。这些新数字图像不属于训练数据，比如可以是测试集中的数据。

In [37]:
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
predictions[0]

1/1 [==============================] - 0s 77ms/step


array([2.7478059e-12, 4.5494024e-14, 3.5039185e-10, 7.7871476e-08,
       1.6366950e-16, 8.8183731e-12, 2.5301948e-20, 9.9999988e-01,
       2.2805076e-11, 3.5321632e-09], dtype=float32)

这个数组中每个索引为i的数字对应数字图像test_digits[0]属于类别i的概率。

第一个测试数字在索引为7时的概率最大（0.99999106，几乎等于1），所以根据我们的模型，这个数字一定是7。

In [38]:
predictions[0].argmax()

7

In [39]:
predictions[0][7]

0.9999999

检查测试标签是否与之一致

In [40]:
test_labels[0]

7

In [41]:
predictions[1]

array([3.3673371e-16, 5.1676281e-08, 1.0000000e+00, 3.9192699e-12,
       3.1839095e-26, 1.2451417e-12, 5.5460405e-13, 5.4851905e-25,
       3.2549121e-13, 1.3869019e-22], dtype=float32)

In [42]:
predictions[1].argmax()

2

In [43]:
predictions[1][2]

1.0

In [44]:
test_labels[1]

2

### 代码清单7　在新数据上评估模型
平均而言，此模型对这种前所未见的数字图像进行分类的效果如何？我们来计算在整个测试集上的平均精度

In [45]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 [==============================] - 1s 1ms/step - loss: 0.0668 - accuracy: 0.9822
test_acc: 0.982200026512146
